In [1]:
import limap
import limap.util.io as limapio
import matplotlib.pyplot as plt
import numpy as np
import plyfile
import torch
import tqdm

# 加载线
lines_path = '../../datasets/jmsd/perspective_v1/lines/finaltracks/'
lines, linetracks = limapio.read_lines_from_input(lines_path)
lines_start = np.array([line.start for line in lines])  # [n, 3]
lines_end = np.array([line.end for line in lines])      # [n, 3]

# 加载点
ply_path = '../../datasets/jmsd/perspective_v1/point_cloud_subsampled_merged.ply'
original_pc = plyfile.PlyData.read(ply_path)
vertices = original_pc['vertex']
coords = np.stack([vertices['x'], vertices['y'], vertices['z']], axis=1)
colors = np.stack([vertices['red'], vertices['green'], vertices['blue']], axis=1) / 255.0


I20250729 20:01:30.868050 136251777540416 io.py:read_folder_linetracks:313] Read linetracks from ../../datasets/jmsd/perspective_v1/lines/finaltracks/...


In [5]:
lines[0].length()

0.6038816562605434

In [3]:
# 在线上采样
lines_points = []
lines_colors = []
for line in lines:
    num_samples = int(line.length() // 0.5 + 10)
    line_vec = line.end - line.start
    line_points = np.array([line.start + (i / (num_samples - 1)) * line_vec for i in range(num_samples)])
    # line_colors = np.tile(np.array([[0.0, 1.0, 0.0]]), (num_samples, 1))  # 绿色
    line_colors = np.random.rand(num_samples, 3)  # 随机颜色
    lines_points.append(line_points)
    lines_colors.append(line_colors)
line_points = np.vstack(lines_points)
line_colors = np.vstack(lines_colors)
print(line_points.shape[0], coords.shape[0])

737998 540723


In [4]:
import os
import open3d as o3d

os.environ["XDG_SESSION_TYPE"] = 'x11'
pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(coords)
# pcd.colors = o3d.utility.Vector3dVector(colors)
pcd.points = o3d.utility.Vector3dVector(np.vstack([coords, line_points]))
pcd.colors = o3d.utility.Vector3dVector(np.vstack([colors, line_colors]))

o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from plyfile import PlyData, PlyElement
# 合并点和颜色
merged_coords = np.vstack([coords, line_points])
merged_colors = np.vstack([colors, line_colors])

# 转换为uint8
merged_colors_uint8 = (merged_colors * 255).astype(np.uint8)

# 构建新的vertex结构
vertex_data = np.zeros(merged_coords.shape[0], dtype=[
    ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
    ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')
])
vertex_data['x'] = merged_coords[:, 0]
vertex_data['y'] = merged_coords[:, 1]
vertex_data['z'] = merged_coords[:, 2]
vertex_data['red'] = merged_colors_uint8[:, 0]
vertex_data['green'] = merged_colors_uint8[:, 1]
vertex_data['blue'] = merged_colors_uint8[:, 2]

# 创建新的PlyElement
vertex_element = PlyElement.describe(vertex_data, 'vertex')

# 保存为新的ply文件
PlyData([vertex_element]).write('../../datasets/jmsd/perspective_v1/merged_with_lines_subsampled.ply')